In [1]:
import os

In [2]:
%pwd

'/home/kalema/Projects/Kidney-Disease-Classification-MLflow-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/kalema/Projects/Kidney-Disease-Classification-MLflow-DVC'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list